In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("../data/freebase15k/train.txt",delimiter="\t", header=None)
test = pd.read_csv("../data/freebase15k/test.txt",delimiter="\t", header=None)
validation = pd.read_csv("../data/freebase15k/valid.txt",delimiter="\t", header=None)
train.columns = ['Header', 'Relation', 'Tail']
test.columns = ['Header', 'Relation', 'Tail']
validation.columns = ['Header', 'Relation', 'Tail']

In [3]:
entities = pd.concat([train.Header, train.Tail,test.Header, test.Tail, validation.Header, validation.Tail])
entities = pd.DataFrame(entities,columns=['ID'])
entities = entities.groupby('ID').ID.agg(['count']).reset_index()

In [4]:
h = pd.DataFrame([], columns=['ID'])
h['ID'] = pd.concat([train.Header, test.Header, validation.Header])
h = h.groupby('ID').ID.agg(['count']).reset_index()


t = pd.DataFrame([], columns=['ID'])
t['ID'] = pd.concat([train.Tail, test.Tail, validation.Tail])
t = t.groupby('ID').ID.agg(['count']).reset_index()

In [5]:
entities['h'] = entities.index.map(lambda x : h.loc[entities.iloc[x].ID == h.ID]['count'].item() if entities.ID.isin(h.ID)[x] else 0 )
entities['t'] = entities.index.map(lambda x : t.loc[entities.iloc[x].ID == t.ID]['count'].item() if entities.ID.isin(t.ID)[x] else 0 )

In [6]:
entities

,ID,count,h,t
0,/m/010016,25,14,11
1,/m/0100mt,32,16,16
2,/m/0102t4,15,10,5
3,/m/0104lr,14,9,5
4,/m/0105y2,22,13,9
...,...,...,...,...
14946,/m/0zqq,10,5,5
14947,/m/0zqq8,19,12,7
14948,/m/0zrlp,20,12,8
14949,/m/0zygc,17,11,6


In [7]:
relations = pd.DataFrame([], columns=['ID'])
relations['ID'] = pd.concat([train.Relation,test.Relation,validation.Relation])
relations = relations.groupby('ID').ID.agg(['count']).reset_index()


In [8]:
relations

,ID,count
0,/american_football/football_coach/coaching_his...,1
1,/american_football/football_coach/coaching_his...,1
2,/american_football/football_coach_position/coa...,1
3,/american_football/football_coach_position/coa...,41
4,/american_football/football_player/current_tea...,1
...,...,...
1340,/visual_art/visual_art_form/artists,40
1341,/visual_art/visual_art_form/artworks,1
1342,/visual_art/visual_art_genre/artworks,1
1343,/visual_art/visual_artist/art_forms,40


In [9]:
test['Head_idx'] = test.index.map(mapper=(lambda x : entities.index[entities.ID == test.Header[x]].tolist()[0]))
test['Tail_idx'] = test.index.map(mapper=(lambda x : entities.index[entities.ID == test.Tail[x]].tolist()[0]))

train['Head_idx'] = train.index.map(mapper=(lambda x : entities.index[entities.ID == train.Header[x]].tolist()[0]))
train['Tail_idx'] = train.index.map(mapper=(lambda x : entities.index[entities.ID == train.Tail[x]].tolist()[0]))

validation['Head_idx'] = validation.index.map(mapper=(lambda x : entities.index[entities.ID == validation.Header[x]].tolist()[0]))
validation['Tail_idx'] = validation.index.map(mapper=(lambda x : entities.index[entities.ID == validation.Tail[x]].tolist()[0]))

train['Relation_idx'] = train.index.map(mapper=(lambda x : relations.index[relations.ID == train.Relation[x]].tolist()[0]))
test['Relation_idx'] = test.index.map(mapper=(lambda x : relations.index[relations.ID == test.Relation[x]].tolist()[0]))
validation['Relation_idx'] = validation.index.map(mapper=(lambda x : relations.index[relations.ID == validation.Relation[x]].tolist()[0]))

train = train.drop(['Header', 'Tail', 'Relation'], axis=1)
test = test.drop(['Header', 'Tail', 'Relation'], axis=1)
validation = validation.drop(['Header', 'Tail', 'Relation'], axis=1)

# train = train.to_numpy()
# test = test.to_numpy()
# validation = validation.to_numpy()

In [ ]:
relations['h_count'] = relations.index.map(lambda x: len(test[test.Relation_idx == x].groupby('Head_idx').agg(['count'])))
relations['t_count'] = relations.index.map(lambda x: len(test[test.Relation_idx == x].groupby('Tail_idx').agg(['count'])))
relations['test_r'] = relations.index.map(lambda x: len(test[test.Relation_idx == x]))

In [ ]:
relations['tph'] = relations.test_r / relations.h_count 
relations['hpt'] = relations.test_r / relations.t_count 

In [ ]:
relations.loc[ (relations['tph'] < 1.5) & (relations['hpt'] < 1.5), 'type'] = 1
relations.loc[ (relations['tph'] >= 1.5) & (relations['hpt'] < 1.5), 'type'] = 2
relations.loc[ (relations['tph'] < 1.5) & (relations['hpt'] >= 1.5), 'type'] = 3
relations.loc[ (relations['tph'] >= 1.5) & (relations['hpt'] >= 1.5), 'type'] = 4


In [ ]:
totalWords = sum([freq[1] **(3/4) for freq in entities.values])     
entities['Probability'] = (entities['count']**(3/4) / totalWords)

totalWords = sum([freq[1] **(3/4) for freq in relations.values])     
relations['Probability'] = (relations['count']**(3/4) / totalWords)

In [ ]:
entities.reset_index(level=0, inplace=True)
entities = entities.drop(['ID'], axis=1)
entities = entities.to_numpy()

relations.reset_index(level=0, inplace=True)
relations = relations.drop(['ID'], axis=1)
relations = relations.to_numpy()

train = train.to_numpy()
test = test.to_numpy()
validation = validation.to_numpy()

In [ ]:
np.save('../data/freebase15k_numpy/train.npy', train)
np.save('../data/freebase15k_numpy/validation.npy', validation)
np.save('../data/freebase15k_numpy/test.npy', test)

np.save('../data/freebase15k_numpy/entities.npy', entities)
np.save('../data/freebase15k_numpy/relations.npy', relations)